<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Postgress_Windows_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install

In [1]:
# Install postgresql server
#!sudo apt-get -y -qq update > /dev/null
!apt update > /dev/null
#!sudo apt-get -y -qq install postgresql > /dev/null
!apt install postgresql > /dev/null
#!sudo service postgresql start
!service postgresql start





 * Starting PostgreSQL 10 database server
   ...done.


## Configure

In [2]:
# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#!sudo -u postgres psql -U postgres -c "ALTER USER tuser2 PASSWORD 'pw2';"
# Setup a database with name `sampledb` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS praxisdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE praxisdb;'

ALTER ROLE
NOTICE:  database "praxisdb" does not exist, skipping
DROP DATABASE
CREATE DATABASE


In [3]:
#!pip install psycopg2
import psycopg2
import pandas as pd

In [4]:
def runSelect(s):
    #Set up connection
    conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(result)
    colnames = [desc[0] for desc in cursor.description]
    #print(colnames)
    resultdf = pd.DataFrame(result)
    resultdf.columns = colnames
    return resultdf

def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

#Windows Functions 
https://www.postgresqltutorial.com/postgresql-window-function/

In [5]:
ddl = "\
CREATE TABLE product_groups (\
	group_id serial PRIMARY KEY,\
	group_name VARCHAR (255) NOT NULL\
);\
\
CREATE TABLE products (\
	product_id serial PRIMARY KEY,\
	product_name VARCHAR (255) NOT NULL,\
	price DECIMAL (11, 2),\
	group_id INT NOT NULL,\
	FOREIGN KEY (group_id) REFERENCES product_groups (group_id)\
);"

runCmd(ddl)

Command executed


In [6]:
ddl = "\
INSERT INTO product_groups (group_name) \
VALUES \
	('Smartphone'), \
	('Laptop'), \
	('Tablet'); \
\
INSERT INTO products (product_name, group_id,price) \
VALUES \
	('Microsoft Lumia', 1, 200),\
	('HTC One', 1, 400),\
	('Nexus', 1, 500),\
	('iPhone', 1, 900),\
	('HP Elite', 2, 1200),\
	('Lenovo Thinkpad', 2, 700),\
	('Sony VAIO', 2, 700),\
	('Dell Vostro', 2, 800),\
	('iPad', 3, 700),\
	('Kindle Fire', 3, 150),\
	('Samsung Galaxy Tab', 3, 200);"

runCmd(ddl)

Command executed


In [7]:
runSelect("select * from product_groups;")

,group_id,group_name
0,1,Smartphone
1,2,Laptop
2,3,Tablet


In [9]:
runSelect("select * from products;")

,product_id,product_name,price,group_id
0,1,Microsoft Lumia,200.00,1
1,2,HTC One,400.00,1
2,3,Nexus,500.00,1
3,4,iPhone,900.00,1
4,5,HP Elite,1200.00,2
5,6,Lenovo Thinkpad,700.00,2
6,7,Sony VAIO,700.00,2
7,8,Dell Vostro,800.00,2
8,9,iPad,700.00,3
9,10,Kindle Fire,150.00,3


In [10]:
sql = "\
SELECT \
	AVG (price) \
FROM \
	products; \
"
runSelect(sql)

,avg
0,586.3636363636363636


In [12]:
sql = "\
SELECT \
	group_name, \
	AVG (price) \
FROM\
	products \
INNER JOIN product_groups USING (group_id) \
GROUP BY \
	group_name;\
"
runSelect(sql)

,group_name,avg
0,Smartphone,500.0000000000000000
1,Tablet,350.0000000000000000
2,Laptop,850.0000000000000000


In [13]:
sql = "\
SELECT\
	product_name,\
	price,\
	group_name,\
	AVG (price) OVER (\
	   PARTITION BY group_name\
	)\
FROM\
	products\
	INNER JOIN \
		product_groups USING (group_id);\
"
runSelect(sql)

,product_name,price,group_name,avg
0,HP Elite,1200.00,Laptop,850.0000000000000000
1,Lenovo Thinkpad,700.00,Laptop,850.0000000000000000
2,Sony VAIO,700.00,Laptop,850.0000000000000000
3,Dell Vostro,800.00,Laptop,850.0000000000000000
4,Microsoft Lumia,200.00,Smartphone,500.0000000000000000
5,HTC One,400.00,Smartphone,500.0000000000000000
6,Nexus,500.00,Smartphone,500.0000000000000000
7,iPhone,900.00,Smartphone,500.0000000000000000
8,iPad,700.00,Tablet,350.0000000000000000
9,Kindle Fire,150.00,Tablet,350.0000000000000000


In [14]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2023-01-17 13:06:54.077627+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)